#turicreate api를 활용한 추천 모델 구현 및 저장
https://apple.github.io/turicreate/docs/api/turicreate.toolkits.recommender.html

####turicreate - 애플에서 만든 오픈 소스 라이브러리.  이미지 분류, 객체 감지, 스타일 전송, 추천 등의 Core ML 모델을 생성할 수 있다.


---
####이 코드에서 구현한 추천모델

 the ranking factorization model : 사용자가 아이템을 높게 평가할지 예측하는 것을 목표로 하는 모델. (추천의 정확도보다는 평가 예측을 중심으로 하는 모델) 각 사용자 및 항목에 대한 잠재 요소를 학습하고 해당(사용자, 항목) 쌍을 관찰할 가능성에 따라 항목의 순위를 지정한다.   <br><br>
 content base model : 콘텐츠 기반 추천. 아이템 내용의 유사성을 사용한다. 이 모델에서 두 항목 간의 유사도 점수는 먼저 각 열에 대한 항목 데이터 간의 유사도를 계산한 후, 컬럼별 유사도의 가중 평균을 취하여 계산된다. 


---
####참고자료
https://ichi.pro/ko/turi-createlo-chucheon-siseutem-guchug-162229068408433
https://apple.github.io/turicreate/docs/userguide/recommender/<br>
*국내 자료 없음. 공식 문서 또는 깃허브 참조.


In [ ]:
#turicreate 설치
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate

In [ ]:
#코사인 유사도 비교를 위해 필요한 라이브러리
import pandas as pd
import numpy as np

#turicreata api
import turicreate as tc

#기타
import re

In [ ]:
#데이터 호출 및 유저 id 생성, 불필요한 피처 제거
data=pd.read_csv('/content/drive/MyDrive/project/MIB/DATA/df_model_real_final.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop('p1_lat', axis=1, inplace=True)
data.drop('p1_log', axis=1, inplace=True)
data.drop('p1_time', axis=1, inplace=True)
data.drop('reached_top', axis=1, inplace=True)
data.drop('sex_age', axis=1, inplace=True)
data.drop('dist/hour', axis=1, inplace=True)
data.drop('season_time', axis=1, inplace=True)
data.drop('speed_median', axis=1, inplace=True)
data.drop('sports_type', axis=1, inplace=True)
data.drop('top_speed', axis=1, inplace=True)

# 중복제거
data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

#컬럼명 변경
data.rename(columns={'name':'start_location'},inplace=True)

#유저 id 생성
data['id']=data['f_name'].apply(lambda x: re.sub(r"[^0-9 ]", "", x))

#레벨 숫자로 변경
data['level']=data['level'].apply(lambda x: re.sub(r"중하", "2", x))
data['level']=data['level'].apply(lambda x: re.sub(r"중상", "4", x))
data['level']=data['level'].apply(lambda x: re.sub(r"하", "1", x))
data['level']=data['level'].apply(lambda x: re.sub(r"중", "3", x))
data['level']=data['level'].apply(lambda x: re.sub(r"상", "5", x))

#수치화
data['level']=pd.to_numeric(data['level'])

#운동 컬럼 추가
exercise=[]
for i in range(len(data)):
    d=data.loc[i]
    if d['level']>=3: 
      exercise.append(1) # 정상 도달
    elif 1 in d[['자운봉','백운대','인수봉','비봉','만경대','향로봉','만장봉','족두리봉','선인봉','오봉']].values.tolist(): 
      exercise.append(1) # 정상 도달
    else:
        exercise.append(0) # 정상 미도달    
data['건강'] = exercise

#힐링 컬럼 추가
heal= []
for i in range(len(data)):
    d=data.loc[i]
    if 1 in d[['송추계곡','사패산/포대능선','사모바위','의상능선','숨은벽','대동문','북한산성','진관사','진흥왕순수비','대남문']].values.tolist(): 
        heal.append(1) # 경관 도달
    else:
        heal.append(0) # 경관 미도달
data['힐링'] = heal

data.to_csv('df_h_e_final.csv')

In [ ]:
data

In [ ]:
#트레킹 코스 정보
trail=data[['f_name','start_location','track_length','lowest_ele','top_ele','elevation_gain','level',	'sub_ele','자운봉',	'백운대',	'인수봉',	'비봉','만경대','향로봉',	'만장봉','족두리봉','선인봉','오봉','송추계곡','사패산/포대능선','사모바위','의상능선','숨은벽','대동문','북한산성','진관사','진흥왕순수비','대남문','건강','힐링']]
td=tc.SFrame(trail)

# 유저 정보
user=data[['id','sex','age']]
ud=tc.SFrame(user)


In [ ]:
#테스트셋/ 데이터셋 분리
d=tc.SFrame(data)
train,test = tc.recommender.util.random_split_by_user(d,user_id='id',item_id='f_name')
print(len(train),len(test))

#recommender.ranking_factorization_recommender 를 사용한 추천 모델 생성 및 저장


In [ ]:
#Factorization machines
ranking_fac_model=tc.recommender.ranking_factorization_recommender.create(d,user_id='id',item_id='f_name',user_data=ud, item_data=td,solver='adagrad')

#폴더 생성하여 모델 저장
ranking_fac_model.save('/content/drive/MyDrive/project/MIB/DATA/hike_ranking_factorization_recommender')

In [ ]:
ranking_fac_model

# item-item similarities based on users 생성 및 저장

In [ ]:
#공통된 유저를 기반으로 하는 아이템 모델 생성
fac_model= tc.item_similarity_recommender.create(train,user_id='id',item_id='f_name')
#폴더 생성하여 저장
fac_model.save('/content/drive/MyDrive/project/MIB/DATA/hike_item_item_model')

#콘텐츠 기반 추천 모델 생성 및 저장

In [ ]:
#관계 정보
od=tc.SFrame(data[['id','f_name']])

In [ ]:
#컨텐츠 기반 모델 생성
content_model=tc.recommender.item_content_recommender.create(td,'f_name',observation_data=od , user_id='id')
#폴더 생성하여 저장
content_model.save('/content/drive/MyDrive/project/MIB/DATA/hike_content_recommender')

###user/item csv 저장

In [ ]:
trail.to_csv('/content/drive/MyDrive/project/MIB/DATA/trail.csv')
user.to_csv('/content/drive/MyDrive/project/MIB/DATA/user.csv')

tr=pd.DataFrame(train)
te=pd.DataFrame(test)
tr.to_csv('/content/drive/MyDrive/project/MIB/DATA/train.csv')
te.to_csv('/content/drive/MyDrive/project/MIB/DATA/test.csv')